In [1]:
from opensky_api import OpenSkyApi
from pyproj import Proj, transform
import io
import urllib.request
from PIL import Image
from math import pi, log, tan, exp, atan, log2, floor
import numpy as np
import pandas as pd
from scipy.spatial import distance
import math
import itertools

import pandas as pd

from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import STAMEN_TERRAIN, STAMEN_TERRAIN_RETINA, CARTODBPOSITRON
from bokeh.models import CheckboxButtonGroup, ColumnDataSource
from bokeh.io import output_notebook, push_notebook
import time

import importlib
moduleName = 'tools'
aspot = importlib.import_module('airplane_spotter')

# bbox = (44, 48, 5, 10) # Central europe
bbox = (47, 48, 8, 9)  # Zurich area (latitudes and longitudes)
# We need to transform lat lon in web mercator coordinates for plotting
x_range, y_range = aspot.bbox2range(bbox)
x_range, y_range = (893463, 1006021), (5900000,6100000)
plot_width = plot_height = 900

os_columns = ['icao24',
              'callsign',
              'origin_country',
              'time_position',
              'longitude',
              'latitude',
              'geo_altitude',
              'on_ground',
              'velocity',
              'heading',
              'sensors',
              'baro_altitude',
              'squawk',
              'spi',
              'position_source']


In [2]:
# 951946, 6005657
outProj = Proj(init='epsg:3857')
inProj = Proj(init='epsg:4326')
y, x = transform(inProj, outProj, 47, 8) 
# x_range, y_range
x, y 

(893463.7510126453, 5232016.067283858)

In [3]:
y, x = transform(inProj, outProj, 48, 9) 
x, y 

(1006021.0627551326, 5343335.558077131)

# Retreive Full flights informations


In [4]:
df = aspot.get_plane_full_data(bbox)
df.dropna(axis=0, subset=['latitude', 'longitude', 'geo_altitude'], inplace=True)
df.head()

,icao24,callsign,origin_country,time_position,longitude,latitude,geo_altitude,on_ground,velocity,heading,sensors,baro_altitude,squawk,spi,position_source
0,4b180b,SWR159E,Switzerland,1563216869,8.4810,47.5224,807.72,False,73.92,137.54,None,723.9,1000,False,0
0,3c662c,EWG6F,Germany,1563216869,8.8263,47.9083,7231.38,False,198.90,353.32,None,7239,3036,False,0
0,4b168e,SWR139J,Switzerland,1563216869,8.2482,47.6656,1920.24,False,97.37,102.2,None,1844.04,5345,False,0
0,4ca5dd,TOM54H,Ireland,1563216869,8.9115,47.1431,11308.1,False,229.66,291.28,None,10972.8,7526,False,0
0,4d00c1,LGL884,Luxembourg,1563216869,8.2733,47.2220,11963.4,False,222.86,300.83,None,11597.6,2764,False,0


# Plot all the measurements in the selected zone

In [5]:
output_notebook()
p = aspot.base_plot(tools='pan, wheel_zoom, reset, hover',
                    plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
p.add_tile(CARTODBPOSITRON)

if len(df)>1:
    df['X'], df['Y'] = tuple(aspot.ll2wm(df.longitude.values, df.latitude.values))
    df['heading'] = df['heading']*np.pi/180
    source = ColumnDataSource(df)
    p.triangle(x='X', y='Y', angle='heading', source=source, size=10 )
show(p)

Loading BokehJS ...

# Get all the flights that are having a close encounter

In [6]:
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper
from bokeh.models import ColumnDataSource, Range1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import ImageURL
from bokeh.io import curdoc, show

dist_close = 100000

hover = HoverTool(tooltips=[('callsign', '@callsign')])

df = aspot.get_close_encounters(bbox, dist_close)
p = aspot.base_plot(tools='pan, wheel_zoom, reset, hover',
                    plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
p.add_tile(CARTODBPOSITRON)

if len(df)>1:
    df['X'], df['Y'] = tuple(aspot.ll2wm(df.longitude.values, df.latitude.values))
    df['heading'] = df['heading']*np.pi/180
    source = ColumnDataSource(df)
    p.triangle(x='X', y='Y', angle='heading', source=source, size=10 )
    
    # We add an airplane image where the planes are
    url = "https://images.sftcdn.net/images/t_app-logo-l,f_auto,dpr_auto/p/41d03908-1e8d-4a18-8eb2-bb0b4dae3360/1593298215/fly-legal-icon.png"
    N = len(df)
    source2 = ColumnDataSource(dict(
        url = [url]*N,
        x1  = df.X,
        y1  = df.Y,
        w1  = [200]*N,
        h1  = [200]*N,
        angle = df.heading
    ))
    image = ImageURL(url="url", x="x1", y="y1", angle="angle", anchor="center")
    p.add_glyph(source2, image)
    
show(p)


C:\Users\figu\Documents\AI\PlaneSpotter\airplane_spotter.py:134: RuntimeWarning: invalid value encountered in less
  close_encounters = np.where(distances < dist_close)


# Now we try to animate a 'real time' view of the sky

In [7]:
target = show(p, notebook_handle=True)
dist_close = 20000
for i in range(10):
    try:
        df = aspot.get_close_encounters(bbox, dist_close)
    except:
        'OpenSkyNetworkAPI not responding'
    if len(df) > 1:
        df['X'], df['Y'] = tuple(aspot.ll2wm(df.longitude.values, df.latitude.values))
        df['heading'] = df['heading']*np.pi/180
        source.stream(df, 0)
        push_notebook(handle=target)


Open Sky Network API is not responding
Open Sky Network API is not responding
Open Sky Network API is not responding
